In [1]:
# default_exp data.dataset_factory

In [2]:
from nbdev.export import *

## Utility Functions for Loading Data from OmegaConf config

In [3]:
# export
import albumentations as A
import torchvision.transforms as T
from hydra.utils import instantiate
from omegaconf import DictConfig, OmegaConf
from torch.utils.data import DataLoader, Dataset

from src import _logger
from src.data.datasets import CassavaDataset, load_data

In [4]:
# export
def create_transform(cfg: DictConfig, verbose=False):
    "creates transoformations to be used in datasets"
    train_augs_initial = [instantiate(t) for t in cfg.train.before_mix]
    train_augs_final   = [instantiate(t) for t in cfg.train.after_mix]
    valid_augs = [instantiate(t) for t in cfg.valid]
    
    if cfg.backend == "torchvision":
        compose_func = T.Compose
    elif cfg.backend == "albumentations":
        compose_func = A.Compose
    
    train_augs_initial = compose_func(train_augs_initial)
    train_augs_final   = compose_func(train_augs_final)
    valid_augs = compose_func(valid_augs)
    return train_augs_initial, train_augs_final, valid_augs

In [5]:
# export
def create_loader(cfg: DictConfig):
    "creates a `DataLoader's instance from Cfg`"
    ds_config = cfg.data.dataset
    tfm_config = cfg.augmentations
    dls_conf = cfg.data.dataloader

    transformations = create_transform(tfm_config)

    df = load_data(ds_config.csv, ds_config.image_dir,ds_config.fold, shuffle=True)
    
    trn_df = df.loc[df["is_valid"] == False]
    val_df = df.loc[df['is_valid'] == True]

    trn_df.reset_index(inplace=True, drop=True)
    val_df.reset_index(inplace=True, drop=True)

    _logger.info("Data processing configuration for current dataset:")
    _logger.info(f"\tinput_size: {cfg.input.input_size}")
    _logger.info(f"\tmean: {tuple(cfg.input.mean)}")
    _logger.info(f"\tstd: {tuple(cfg.input.std)}")
    _logger.info(f"\tinterpolation: {cfg.input.interpolation}")
    _logger.info("")

    trn_tfm, tfn_tfm_after, val_tfm = create_transform(tfm_config)

    if tfm_config.backend == "torchvision":
        trn_dset = CassavaDataset.from_torchvision_tfms(trn_df, fn_col="filePath",
                                                        label_col="label", transform=trn_tfm)

        val_dset = CassavaDataset.from_torchvision_tfms(val_df, fn_col="filePath",
                                                        label_col="label", transform=val_tfm)

    elif tfm_config.backend == "albumentations":
        trn_dset = CassavaDataset.from_albu_tfms(trn_df, fn_col="filePath",
                                                 label_col="label", transform=trn_tfm)

        val_dset = CassavaDataset.from_albu_tfms(val_df, fn_col="filePath",
                                                 label_col="label", transform=val_tfm)
        
    train_dl = DataLoader(trn_dset, shuffle=True, **dls_conf)
    val_dl = DataLoader(val_dset, shuffle=True, **dls_conf)
    
    return train_dl, val_dl, trn_dset, tfn_tfm_after

In [6]:
from hydra.experimental import initialize, compose
from src.models.builder import Net

with initialize("../conf/"):
    cfg = compose("effnet-base", overrides=['general=default'])

In [7]:
mixmethod = instantiate(cfg.mixmethod)
train_dl, val_dl, train_dset, tfms = create_loader(cfg)

_logger.info(f"Mixmethod: {mixmethod}")
model = Net(cfg)

image, target = next(iter(train_dl))
image_1 = mixmethod(image, target, model)

mixmethod.stop()
train_dset.reload_transforms(tfms)
train_dl = DataLoader(train_dset, shuffle=True, **cfg.data.dataloader)

image, target = next(iter(train_dl))
image_2 = mixmethod(image, target, model)

[01/31 06:40:44 __main__]: Data processing configuration for current dataset:
[01/31 06:40:44 __main__]: 	input_size: 512
[01/31 06:40:44 __main__]: 	mean: (0.42984136, 0.49624753, 0.3129598)
[01/31 06:40:44 __main__]: 	std: (0.21417203, 0.21910103, 0.19542212)
[01/31 06:40:44 __main__]: 	interpolation: random
[01/31 06:40:44 __main__]: 
[01/31 06:40:45 __main__]: Mixmethod: Snapmix(alpha=5.0, conf_prob=0.5, num_iters=17)
[01/31 06:40:45 src.models.builder]: Configuration for the current model :
[01/31 06:40:45 src.models.builder]: 	 base_network: tf_efficientnet_b3_ns
[01/31 06:40:45 src.models.builder]: 	 activation: mish
[01/31 06:40:45 src.models.builder]: 	 drop_path_rate: 0.25
[01/31 06:40:45 src.models.builder]: Configuration for model head : 
[01/31 06:40:45 src.models.builder]: 	 class_name: CnnHeadV0
[01/31 06:40:45 src.models.builder]: 	 n_out: 5
[01/31 06:40:45 src.models.builder]: 	 pool_type: avg
[01/31 06:40:45 src.models.builder]: 	 use_conv: False
[01/31 06:40:45 src.m

In [8]:
notebook2script()

Converted 00_core.ipynb.
Converted 01_data.datasets.ipynb.
Converted 01a_data.mixmethods.ipynb.
Converted 01b_data.datasests_factory.ipynb.
Converted 02_losses.ipynb.
Converted 03_models.utils.ipynb.
Converted 03a_models.builder.ipynb.
Converted 03a_models.layers.ipynb.
Converted 03b_models.classifiers.ipynb.
Converted 04_optimizers.ipynb.
Converted 04a_schedulers.ipynb.
Converted 05a_lightning.core.ipynb.
Converted 05b_lightning.callbacks.ipynb.
Converted 05c_lightning.callbacks.progress.ipynb.
Converted 06_fastai.core.ipynb.
Converted index.ipynb.
